In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import pickle

In [2]:
# 定义一个时间拆分函数
def split_date(data: DataFrame, col: str):
    """
    拆分时间
    :param data:需要拆分的数据
    :param col: 时间所在的列
    :return: DataFrame
    """
    data['year'] = data[col].apply(lambda x: x.year)
    data['month'] = data[col].apply(lambda x: x.month)
    data['day'] = data[col].apply(lambda x: x.day)
    data['hour'] = data[col].apply(lambda x: x.hour)
    return data

In [3]:
# 读取train
train = pd.read_csv('../data/train.csv', parse_dates=['datetime'])
train = split_date(train, 'datetime')    # 拆分时间
train[train['target'].isna()]['day'].unique()

array([31, 27, 30, 26], dtype=int64)

# 拼接历史target(即两天前的target)

In [4]:
train_history = train.copy()
# 将日期增加两天，因为官网上测试集历史target和要预测的target差两天
train_history['datetime'] = train_history['datetime'].apply(lambda x: x + pd.Timedelta(2, 'D'))
train_history.rename(columns={'target': 'target_used'}, inplace=True)
train_history.drop(columns=['data_block_id', 'row_id', 'prediction_unit_id', 'year', 'month', 'day', 'hour'], inplace=True)
train = pd.merge(left=train, right=train_history, how='left', on=['datetime', 'county', 'is_business', 'product_type', 
                                                                  'is_consumption'])

# 拼接train和client

In [5]:
# 读取client
client = pd.read_csv('../data/client.csv', parse_dates=['date'])
client['datetime'] = client['date'].apply(lambda x: x+pd.Timedelta(2, 'D'))  # 将day向前移两天
client = split_date(client, 'datetime')
client.drop(columns=['hour', 'datetime', 'date', 'data_block_id'], inplace=True)
# 拼接train和client
train_client = pd.merge(left=train, right=client, how='left' ,on=['product_type', 'county', 'is_business', 'year', 'month', 'day'])
train_client

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,year,month,day,hour,target_used,eic_count,installed_capacity
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021,9,1,0,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021,9,1,0,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021,9,1,0,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021,9,1,0,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021,9,1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023,5,31,23,188.167,15.0,620.0
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023,5,31,23,0.000,20.0,624.5
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023,5,31,23,31.484,20.0,624.5
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023,5,31,23,0.000,55.0,2188.2


# 拼接gas_prices

In [6]:
# 读取天然气价格
gas_prices = pd.read_csv('../data/gas_prices.csv', parse_dates=['forecast_date'])
gas_prices

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4
4,2021-09-05,46.30,46.58,2021-09-04,5
...,...,...,...,...,...
632,2023-05-26,29.10,34.10,2023-05-25,633
633,2023-05-27,28.30,34.10,2023-05-26,634
634,2023-05-28,28.10,34.10,2023-05-27,635
635,2023-05-29,28.16,36.98,2023-05-28,636


In [7]:
# 位移一天与train匹配
gas_prices['datetime'] = gas_prices['forecast_date'].apply(lambda x: x + pd.Timedelta(1, 'D'))
gas_prices = split_date(gas_prices, 'datetime')
gas_prices.drop(columns=['origin_date', 'datetime', 'forecast_date', 'hour', 'data_block_id'], inplace=True)
# 与总数据拼接
train_client_gas = pd.merge(left=train_client, right=gas_prices, how='left', on=['year', 'month', 'day'])
train_client_gas

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,year,month,day,hour,target_used,eic_count,installed_capacity,lowest_price_per_mwh,highest_price_per_mwh
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021,9,1,0,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021,9,1,0,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021,9,1,0,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021,9,1,0,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021,9,1,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023,5,31,23,188.167,15.0,620.0,29.0,34.0
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023,5,31,23,0.000,20.0,624.5,29.0,34.0
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023,5,31,23,31.484,20.0,624.5,29.0,34.0
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023,5,31,23,0.000,55.0,2188.2,29.0,34.0


# 拼接electricity_prices

In [8]:
electricity = pd.read_csv('../data/electricity_prices.csv', parse_dates=['forecast_date'])
electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1
...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637


In [9]:
# 位移一天，和train匹配
electricity['datetime'] = electricity['forecast_date'].apply(lambda x: x + pd.Timedelta(1, 'D'))
electricity.drop(columns=['forecast_date', 'origin_date', 'data_block_id'], inplace=True)
train_client_gas_elc = pd.merge(left=train_client_gas, right=electricity,how='left', on='datetime')
train_client_gas_elc

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,year,month,day,hour,target_used,eic_count,installed_capacity,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021,9,1,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021,9,1,0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021,9,1,0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021,9,1,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021,9,1,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023,5,31,23,188.167,15.0,620.0,29.0,34.0,-1.29
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023,5,31,23,0.000,20.0,624.5,29.0,34.0,-1.29
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023,5,31,23,31.484,20.0,624.5,29.0,34.0,-1.29
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023,5,31,23,0.000,55.0,2188.2,29.0,34.0,-1.29


# 拼接historical_weather

In [10]:
historical_weather = pd.read_csv("../data/historical_weather.csv",parse_dates=['datetime'])
# 由于historical_weather的datetime与train错位37个小时, 因此将historical_weather的datetime加37个小时
historical_weather['datetime'] = historical_weather['datetime'].apply(lambda x: x+pd.Timedelta(37, 'H'))
# 将经纬度取一位小数
historical_weather['latitude'] = historical_weather['latitude'].round(1)
historical_weather['longitude'] = historical_weather['longitude'].round(1)
historical_weather

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-02 13:00:00,14.4,12.0,0.0,0.0,1015.8,4,4,0,0,6.694444,3,0.0,0.0,0.0,57.6,21.7,1
1,2021-09-02 13:00:00,14.0,12.0,0.0,0.0,1010.6,7,8,0,0,4.944444,353,0.0,0.0,0.0,57.6,22.2,1
2,2021-09-02 13:00:00,14.4,12.8,0.0,0.0,1014.9,6,7,0,0,5.833333,348,0.0,0.0,0.0,57.6,22.7,1
3,2021-09-02 13:00:00,15.4,13.0,0.0,0.0,1014.4,4,2,4,0,7.111111,349,0.0,0.0,0.0,57.6,23.2,1
4,2021-09-02 13:00:00,15.9,12.6,0.0,0.0,1013.8,12,7,0,20,8.388889,360,0.0,0.0,0.0,57.6,23.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710795,2023-05-31 23:00:00,10.8,4.1,0.4,0.0,1018.7,40,0,67,0,1.333333,283,440.0,264.0,176.0,59.7,26.2,637
1710796,2023-05-31 23:00:00,11.4,2.9,0.3,0.0,1018.8,41,0,69,0,1.527778,302,453.0,283.0,170.0,59.7,26.7,637
1710797,2023-05-31 23:00:00,9.5,3.2,0.1,0.0,1018.9,47,0,79,0,3.416667,302,477.0,312.0,165.0,59.7,27.2,637
1710798,2023-05-31 23:00:00,11.0,1.6,0.0,0.0,1018.6,31,0,52,0,3.472222,309,535.0,400.0,135.0,59.7,27.7,637


In [11]:
station = pd.read_csv('../data/weather_station_to_county_mapping.csv')
# 将经纬度取一位小数
station['longitude'] = station['longitude'].round(1)
station['latitude'] = station['latitude'].round(1)

In [12]:
# 与station数据按照经纬度和时间拼接
historical_weather = pd.merge(left=historical_weather, right=station, how='left', on=['latitude', 'longitude'])
historical_weather.dropna(subset='county', inplace=True)
historical_weather.head()

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,county_name,county
10,2021-09-02 13:00:00,13.4,11.8,0.1,0.0,998.5,100,92,48,99,3.500000,358,0.0,0.0,0.0,57.6,26.7,1,Võrumaa,15.0
11,2021-09-02 13:00:00,13.1,12.3,0.3,0.0,988.9,100,100,81,100,3.305556,3,0.0,0.0,0.0,57.6,27.2,1,Võrumaa,15.0
23,2021-09-02 13:00:00,13.6,11.1,0.0,0.0,1005.6,40,16,0,86,3.722222,354,0.0,0.0,0.0,57.9,26.2,1,Valgamaa,13.0
24,2021-09-02 13:00:00,13.7,11.2,0.0,0.0,998.0,69,45,0,95,3.611111,358,0.0,0.0,0.0,57.9,26.7,1,Võrumaa,15.0
25,2021-09-02 13:00:00,13.6,12.0,0.0,0.0,999.6,100,81,7,96,3.611111,360,0.0,0.0,0.0,57.9,27.2,1,Võrumaa,15.0


In [13]:
historical_weather.drop(columns=['county_name', 'data_block_id', 'latitude', 'longitude'], inplace=True)
# 由于多个经纬度对应一个county，一个时间点一个county会有几条数据，对他们取均值
historical_weather_gb = historical_weather.groupby(by=['datetime', 'county']).mean()

In [14]:
train_client_gas_elc_hs = pd.merge(left=train_client_gas_elc, right=historical_weather_gb, how='left', on=['datetime', 'county'])
train_client_gas_elc_hs

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,year,...,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023,...,1005.08,40.4,7.0,57.0,0.0,1.938889,327.2,394.6,231.8,162.8
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023,...,1005.08,40.4,7.0,57.0,0.0,1.938889,327.2,394.6,231.8,162.8
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023,...,1005.08,40.4,7.0,57.0,0.0,1.938889,327.2,394.6,231.8,162.8
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023,...,1005.08,40.4,7.0,57.0,0.0,1.938889,327.2,394.6,231.8,162.8


# 拼接forecast_weather

In [15]:
# 读取预测天气数据
forecast_weather = pd.read_csv('../data/forecast_weather.csv', parse_dates=['forecast_datetime'])
# 选取预测小时数大于24小时的数据
forecast_weather = forecast_weather[forecast_weather['hours_ahead'] >= 24]
forecast_weather.head()

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
2576,57.6,21.7,2021-09-01 00:00:00+00:00,24,13.850854,7.572046,0.0,0.066940,0.649811,0.657730,1.278297,-10.211323,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000039
2577,57.6,22.2,2021-09-01 00:00:00+00:00,24,12.025049,5.299829,0.0,0.153427,0.373154,0.396469,0.983253,-5.645649,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000002
2578,57.6,22.7,2021-09-01 00:00:00+00:00,24,13.106104,6.868921,0.0,0.144882,0.143845,0.206787,1.875343,-7.886372,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000013
2579,57.6,23.2,2021-09-01 00:00:00+00:00,24,13.237939,7.634546,0.0,0.185043,0.219177,0.277496,1.476051,-9.236713,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000062
2580,57.6,23.7,2021-09-01 00:00:00+00:00,24,13.522729,7.276147,0.0,0.176743,0.233383,0.285309,1.917457,-9.261860,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000061


In [16]:
forecast_weather['latitude'] = forecast_weather['latitude'].round(1)
forecast_weather['longitude'] = forecast_weather['longitude'].round(1)
forecast_weather.head()

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
2576,57.6,21.7,2021-09-01 00:00:00+00:00,24,13.850854,7.572046,0.0,0.066940,0.649811,0.657730,1.278297,-10.211323,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000039
2577,57.6,22.2,2021-09-01 00:00:00+00:00,24,12.025049,5.299829,0.0,0.153427,0.373154,0.396469,0.983253,-5.645649,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000002
2578,57.6,22.7,2021-09-01 00:00:00+00:00,24,13.106104,6.868921,0.0,0.144882,0.143845,0.206787,1.875343,-7.886372,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000013
2579,57.6,23.2,2021-09-01 00:00:00+00:00,24,13.237939,7.634546,0.0,0.185043,0.219177,0.277496,1.476051,-9.236713,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000062
2580,57.6,23.7,2021-09-01 00:00:00+00:00,24,13.522729,7.276147,0.0,0.176743,0.233383,0.285309,1.917457,-9.261860,1,2021-09-02 00:00:00+00:00,0.0,0.0,0.0,0.000061


In [17]:
# 按照经纬度和station合并，获取county信息
forecast_weather = pd.merge(left=forecast_weather, right=station, on=['latitude', 'longitude'])

In [18]:
# 去除缺失值以及删除无用列
forecast_weather.dropna(subset='county', inplace=True)
forecast_weather.drop(columns=['county_name', 'latitude', 'longitude', 'origin_datetime', 'hours_ahead', 'data_block_id'], inplace=True)

In [19]:
# 将forecast列索引重命名，以防止合并后与historical重名
forecast_cols_new = {}
forecast_cols = forecast_weather.columns
for index in forecast_cols:
    if index == 'forecast_datetime':
        index_new = 'datetime'
    elif index == 'county':
        index_new = index
    else:
        index_new = str(index) + '_fw'
    forecast_cols_new[index] = index_new
forecast_weather.rename(columns=forecast_cols_new, inplace=True)
forecast_weather.columns

Index(['temperature_fw', 'dewpoint_fw', 'cloudcover_high_fw',
       'cloudcover_low_fw', 'cloudcover_mid_fw', 'cloudcover_total_fw',
       '10_metre_u_wind_component_fw', '10_metre_v_wind_component_fw',
       'datetime', 'direct_solar_radiation_fw',
       'surface_solar_radiation_downwards_fw', 'snowfall_fw',
       'total_precipitation_fw', 'county'],
      dtype='object')

In [20]:
# 去除时间UTC值
forecast_weather['datetime'] = pd.to_datetime(forecast_weather.datetime).dt.tz_localize(None)
forecast_weather

,temperature_fw,dewpoint_fw,cloudcover_high_fw,cloudcover_low_fw,cloudcover_mid_fw,cloudcover_total_fw,10_metre_u_wind_component_fw,10_metre_v_wind_component_fw,datetime,direct_solar_radiation_fw,surface_solar_radiation_downwards_fw,snowfall_fw,total_precipitation_fw,county
159250,8.488062,5.375024,0.000000,0.118958,0.001358,0.119446,2.307227,-2.966694,2021-09-02 00:00:00,0.000000,0.000000,0.0,0.0,15.0
159251,8.049463,5.473657,0.000000,0.194870,0.020065,0.197113,2.158781,-2.947390,2021-09-02 01:00:00,0.000000,0.000000,0.0,0.0,15.0
159252,7.881982,5.721948,0.000000,0.056168,0.002319,0.056458,2.248640,-2.831182,2021-09-02 02:00:00,0.000000,0.095556,0.0,0.0,15.0
159253,7.473413,5.835229,0.000000,0.056274,0.008728,0.057648,2.254070,-2.612130,2021-09-02 03:00:00,0.000000,0.000000,0.0,0.0,15.0
159254,7.642480,5.864893,0.000000,0.133194,0.027618,0.135193,2.235337,-2.686131,2021-09-02 04:00:00,76.328056,12.515278,0.0,0.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544720,12.861963,5.246729,0.000000,0.063446,0.047470,0.068604,5.197765,-0.749062,2023-05-31 20:00:00,0.000000,0.000000,0.0,0.0,2.0
1544721,11.828882,5.092920,0.991058,0.093369,0.184280,0.992249,5.351963,-1.289851,2023-05-31 21:00:00,0.404444,0.000000,0.0,0.0,2.0
1544722,11.063013,5.833643,0.995270,0.176430,0.322678,0.996887,4.613072,-1.276831,2023-05-31 22:00:00,0.000000,0.000000,0.0,0.0,2.0
1544723,10.882471,6.015771,0.995575,0.233864,0.502289,0.997803,4.598775,-1.224700,2023-05-31 23:00:00,0.000000,0.000000,0.0,0.0,2.0


In [21]:
# 由于一个county对应多个天气站点，将同一个county同一时间的数据平均
forecast_weather = forecast_weather.groupby(by=['datetime', 'county']).mean()
forecast_weather

temperature_fw  dewpoint_fw  cloudcover_high_fw  \
datetime   county                                                    
2021-09-02 0.0          10.124475     5.054346            0.000000   
           1.0          11.124536     4.258447            0.000000   
           2.0           9.874536     6.938898            0.000000   
           3.0           8.933008     5.559595            0.000000   
           4.0           8.708276     5.914168            0.000000   
...                           ...          ...                 ...   
2023-06-01 10.0          9.703821     5.238824            0.002029   
           11.0          9.262720     4.605676            0.475815   
           13.0          8.815088     3.881494            0.723846   
           14.0          8.382593     3.477319            0.294749   
           15.0          8.790161     4.069385            0.800964   

                   cloudcover_low_fw  cloudcover_mid_fw  cloudcover_total_fw  \
datetime   county                                                              
2021-09-02 0.0              0.116028           0.033717             0.126729   
           1.0              0.036133           0.141724             0.158936   
           2.0              0.348904           0.303078             0.480247   
           3.0              0.292857           0.087118             0.329951   
           4.0              0.129623           0.120346             0.210953   
...                              ...                ...                  ...   
2023-06-01 10.0             0.000965           0.000572             0.003067   
           11.0             0.003971           0.000000             0.479546   
           13.0             0.000000           0.000000             0.723846   
           14.0             0.000000           0.000000             0.294749   
           15.0             0.000000           0.000000             0.800964   

                   10_metre_u_wind_component_fw  10_metre_v_wind_component_fw  \
datetime   county                                                               
2021-09-02 0.0                         1.586301                     -4.571267   
           1.0                         1.360084                     -4.276020   
           2.0                         1.709998                     -4.506062   
           3.0                         2.217953                     -3.215717   
           4.0                         2.515561                     -3.356017   
...                                         ...                           ...   
2023-06-01 10.0                        2.899232                     -2.433590   
           11.0                        4.452760                     -1.488644   
           13.0                        3.747071                     -1.961269   
           14.0                        3.892742                     -2.039598   
           15.0                        3.683009                     -0.604116   

                   direct_solar_radiation_fw  \
datetime   county                              
2021-09-02 0.0                           0.0   
           1.0                           0.0   
           2.0                           0.0   
           3.0                           0.0   
           4.0                           0.0   
...                                      ...   
2023-06-01 10.0                          0.0   
           11.0                          0.0   
           13.0                          0.0   
           14.0                          0.0   
           15.0                          0.0   

                   surface_solar_radiation_downwards_fw  snowfall_fw  \
datetime   county                                                      
2021-09-02 0.0                                      0.0          0.0   
           1.0                                      0.0          0.0   
           2.0                                      0.0          0.0   
           3.0              

In [22]:
# 将总数据与forecas_weather拼接
train_client_gas_elc_hs_fw = pd.merge(left=train_client_gas_elc_hs, right=forecast_weather, how='left', on=['datetime', 'county'])
train_client_gas_elc_hs_fw

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,year,...,cloudcover_high_fw,cloudcover_low_fw,cloudcover_mid_fw,cloudcover_total_fw,10_metre_u_wind_component_fw,10_metre_v_wind_component_fw,direct_solar_radiation_fw,surface_solar_radiation_downwards_fw,snowfall_fw,total_precipitation_fw
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023,...,0.707452,0.0,0.0,0.707452,3.609517,-0.371404,0.0,0.0,0.0,0.0
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023,...,0.707452,0.0,0.0,0.707452,3.609517,-0.371404,0.0,0.0,0.0,0.0
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023,...,0.707452,0.0,0.0,0.707452,3.609517,-0.371404,0.0,0.0,0.0,0.0
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023,...,0.707452,0.0,0.0,0.707452,3.609517,-0.371404,0.0,0.0,0.0,0.0


# 数据处理

In [23]:
# 查看缺失值
train_client_gas_elc_hs_fw.isnull().sum()

county                                      0
is_business                                 0
product_type                                0
target                                    528
is_consumption                              0
datetime                                    0
data_block_id                               0
row_id                                      0
prediction_unit_id                          0
year                                        0
month                                       0
day                                         0
hour                                        0
target_used                              9168
eic_count                                8640
installed_capacity                       8640
lowest_price_per_mwh                     2928
highest_price_per_mwh                    2928
euros_per_mwh                            3196
temperature                             35064
dewpoint                                35064
rain                              

In [24]:
# 去除缺失值
train_client_gas_elc_hs_fw.dropna(how='any', inplace=True)
# 查看列
train_client_gas_elc_hs_fw.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id', 'year',
       'month', 'day', 'hour', 'target_used', 'eic_count',
       'installed_capacity', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'euros_per_mwh', 'temperature', 'dewpoint', 'rain', 'snowfall',
       'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'temperature_fw', 'dewpoint_fw',
       'cloudcover_high_fw', 'cloudcover_low_fw', 'cloudcover_mid_fw',
       'cloudcover_total_fw', '10_metre_u_wind_component_fw',
       '10_metre_v_wind_component_fw', 'direct_solar_radiation_fw',
       'surface_solar_radiation_downwards_fw', 'snowfall_fw',
       'total_precipitation_fw'],
      dtype='object')

In [25]:
# 删除不必要的列
train_client_gas_elc_hs_fw.drop(columns=['datetime', 'data_block_id', 'row_id', 'prediction_unit_id'], inplace=True)
# 对类别变量进行one-hot编码
train_client_gas_elc_hs_fw = pd.get_dummies(train_client_gas_elc_hs_fw, columns=['is_business', 'product_type', 'is_consumption'], dtype=float)

In [26]:
train_client_gas_elc_hs_fw.columns

Index(['county', 'target', 'year', 'month', 'day', 'hour', 'target_used',
       'eic_count', 'installed_capacity', 'lowest_price_per_mwh',
       'highest_price_per_mwh', 'euros_per_mwh', 'temperature', 'dewpoint',
       'rain', 'snowfall', 'surface_pressure', 'cloudcover_total',
       'cloudcover_low', 'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'temperature_fw', 'dewpoint_fw',
       'cloudcover_high_fw', 'cloudcover_low_fw', 'cloudcover_mid_fw',
       'cloudcover_total_fw', '10_metre_u_wind_component_fw',
       '10_metre_v_wind_component_fw', 'direct_solar_radiation_fw',
       'surface_solar_radiation_downwards_fw', 'snowfall_fw',
       'total_precipitation_fw', 'is_business_0', 'is_business_1',
       'product_type_0', 'product_type_1', 'product_type_2', 'product_type_3',
       'is_consumption_0', 'is_consumption_1'],
      dtype='object')

In [27]:
# 生成np.array数组
X = train_client_gas_elc_hs_fw.drop(columns=['target'])
Y = train_client_gas_elc_hs_fw['target']
X = np.array(X)
Y = np.array(Y)

In [28]:
# 生成数据文件
with open('../data/train_data.pkl', 'wb') as f:
    pickle.dump((X, Y), f)

打开方式：  
with open('../data/train_data.pkl', 'rb') as f:  
&emsp;X, Y = pickle.load(f)

In [30]:
# 根据之前的数据分析选择特征
selected_features = ['county', 'year', 'month', 'day', 'hour', 'target_used', 'eic_count', 'installed_capacity', 
                    'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 'temperature', 'dewpoint', 
                    'snowfall', 'cloudcover_total', 'cloudcover_low', 'shortwave_radiation', 'direct_solar_radiation',
                    'diffuse_radiation', 'temperature_fw', 'dewpoint_fw', 'cloudcover_low_fw', 'cloudcover_total_fw',
                    'direct_solar_radiation_fw', 'surface_solar_radiation_downwards_fw', 'snowfall_fw',
                    'is_business_0', 'is_business_1', 'product_type_0', 'product_type_1', 'product_type_2', 'product_type_3',
                    'is_consumption_0', 'is_consumption_1']
X_selected = train_client_gas_elc_hs_fw[selected_features]
Y_selected = train_client_gas_elc_hs_fw['target']
X_selected = np.array(X_selected)
Y_selected = np.array(Y_selected)
# 生成数据文件
with open('../data/train_data_selected.pkl', 'wb') as f:
    pickle.dump((X_selected, Y_selected), f)